# Booking hotel crawler

In [12]:
# import lib
from datetime import datetime, timedelta, date
import requests
from bs4 import BeautifulSoup
import re
import time

最多查詢到明年年底

In [13]:
# Settings
surveyRange = 60
travelPeriod = 4

In [14]:
# Date List 
# args: start date, survey range, travel period(days)
# ex. [datetime.date(2017, 8, 7), datetime.date(2017, 8, 11)] ~ [datetime.date(2017, 9, 5), datetime.date(2017, 9, 9)]]
def getDateList(dt = date.today(), ran = surveyRange, period = travelPeriod):
    timeList = []
    start = dt
    end = start + timedelta(days = period)
    for i in range(1, ran+1):  # i = 1 ~ ran
        timeList.append([start, end])
        start = start + timedelta(days = 1)
        end = end + timedelta(days = 1) 
    return timeList
timeList = getDateList()

In [15]:
# Date List components
checkinM = []
checkinD = []
checkinY = []
checkoutM = []
checkoutD = []
checkoutY = []
for i in timeList:
    checkinM.append(i[0].month)
    checkinD.append(i[0].day)
    checkinY.append(i[0].year)
    checkoutM.append(i[1].month)
    checkoutD.append(i[1].day)
    checkoutY.append(i[1].year)

In [16]:
# headers
# cookies
# location
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'}
cookie = {"Cookie":"zz_cook_tms_seg1=2; zz_cook_tms_seg3=8; header_signin_prompt=1; _ga=GA1.2.771830251.1501676071; _gid=GA1.2.1896765923.1502154343; BJS=-; utag_main=v_id:015da2de69b200b09a95a293b1480407200b206a00bd0$_sn:6$_ss:0$_st:1502156157813$4split:0$4split2:2$ses_id:1502154343183%3Bexp-session$_pn:2%3Bexp-session; zz_cook_tms_ed=1; has_preloaded=1; lastSeen=0; bkng=11UmFuZG9tSVYkc2RlIyh9YfDNyGw8J7nzPnUG3Pr%2Bfv65T5ETw9fe%2BKe5qymT1Z40gA%2FH%2BULT6bEDZOF2KIg5v8IV4jmbGhjcN5s1B29a2EplRUaDxxwLbjCLsAacIQhllRJv09KyPI0yDHlTyUg1RHg7mc46PW1QhGDbReMwEVveo5NHRP6UayKUFO2EkOIJSOyf5zVXk0hk1LXloa439w%3D%3D; _gat=1"}
location = '京都'

In [17]:
# Pages amount for each travel period.  
# ex. numHotelList = [ ['145'], ['160'], ['167'], ['203'], ['239'], ['294'], ['353'], ['371'], ... increasing by date to date

pagesList = []
numHotelList = []
turnPageList = []
for i in range(1, surveyRange+1): # i = 1 ~ surveyRange
    # url位址為XHR文件中的searchresults，去除AJAX部分
    url = "https://www.booking.com/searchresults.zh-tw.html?aid=304142&label=gen173nr-1DCAEoggJCAlhYSDNiBW5vcmVmaOcBiAEBmAEwuAEGyAEM2AED6AEBkgIBeagCAw&sid=25f3195cbf76726e8ffac861269a0d24&checkin_month="+str(checkinM[i-1])+"&checkin_monthday="+str(checkinD[i-1])+"&checkin_year="+str(checkinY[i-1])+"&checkout_month="+str(checkoutM[i-1])+"&checkout_monthday="+str(checkoutD[i-1])+"&checkout_year="+str(checkoutY[i-1])+"&class_interval=1&dest_id=-235402&dest_type=city&group_adults=2&group_children=0&label_click=undef&no_rooms=1&raw_dest_type=city&room1=A%2CA&sb_price_type=total&src=index&src_elem=sb&ss="+str(location)+"&ssb=empty&ssne="+str(location)+"&ssne_untouched="+str(location)+"&rows=50"###+"&sr_ajax=1&b_gtt=dLYAeZFVJfNTBdAHUdPHUBSSUVJfcbWYaNLDRCAET&_=1501050897864"
    res = requests.get(url, headers = headers, cookies = cookie)
    soup = BeautifulSoup(res.text, 'lxml')
    
    # 找總頁數
    clue1 = soup.select('#right')[0].find(lambda x: x.get('data-et-view') == 'TULQWNVLBLWIJDAOfUYdXaO:1')
    pattern = "找到\s(\d+)\s間" #找頁數用
    numHotelText = re.findall(pattern, clue1.find('h1').text)
    # 這區間共幾間hotel
    numHotel = int(numHotelText[0])
    # numHotel / 一頁有50筆  =  翻頁數
    turnPage = round(numHotel/50)
    # append to lists
    pagesList.append(numHotel)
    turnPageList.append(turnPage)

In [18]:
pagesList

[133,
 141,
 159,
 160,
 197,
 243,
 293,
 356,
 373,
 508,
 556,
 598,
 708,
 747,
 753,
 706,
 718,
 735,
 759,
 863,
 876,
 853,
 766,
 773,
 790,
 788,
 860,
 890,
 869,
 799,
 794,
 805,
 831,
 896,
 881,
 831,
 601,
 554,
 557,
 554,
 663,
 852,
 840,
 775,
 796,
 797,
 797,
 871,
 850,
 814,
 719,
 662,
 646,
 598,
 611,
 590,
 547,
 434,
 432,
 461]

In [19]:
turnPageList

[3,
 3,
 3,
 3,
 4,
 5,
 6,
 7,
 7,
 10,
 11,
 12,
 14,
 15,
 15,
 14,
 14,
 15,
 15,
 17,
 18,
 17,
 15,
 15,
 16,
 16,
 17,
 18,
 17,
 16,
 16,
 16,
 17,
 18,
 18,
 17,
 12,
 11,
 11,
 11,
 13,
 17,
 17,
 16,
 16,
 16,
 16,
 17,
 17,
 16,
 14,
 13,
 13,
 12,
 12,
 12,
 11,
 9,
 9,
 9]

In [20]:
# for each period (ex. 10/7 ~ 10/11), grab the hotelList information
start_time = time.time()
for i in range(1, surveyRange+1): # i = 1 ~ surveyRange
    url = "https://www.booking.com/searchresults.zh-tw.html?aid=304142&label=gen173nr-1DCAEoggJCAlhYSDNiBW5vcmVmaOcBiAEBmAEwuAEGyAEM2AED6AEBkgIBeagCAw&sid=25f3195cbf76726e8ffac861269a0d24&checkin_month="+str(checkinM[i-1])+"&checkin_monthday="+str(checkinD[i-1])+"&checkin_year="+str(checkinY[i-1])+"&checkout_month="+str(checkoutM[i-1])+"&checkout_monthday="+str(checkoutD[i-1])+"&checkout_year="+str(checkoutY[i-1])+"&class_interval=1&dest_id=-235402&dest_type=city&group_adults=2&group_children=0&label_click=undef&no_rooms=1&raw_dest_type=city&room1=A%2CA&sb_price_type=total&src=index&src_elem=sb&ss="+str(location)+"&ssb=empty&ssne="+str(location)+"&ssne_untouched="+str(location)+"&rows=50"###+"&sr_ajax=1&b_gtt=dLYAeZFVJfNTBdAHUdPHUBSSUVJfcbWYaNLDRCAET&_=1501050897864"
    res = requests.get(url, headers = headers, cookies = cookie)
    soup = BeautifulSoup(res.text, 'lxml')
    print("=="*50)
    print("period: " + str(i))
    for page in range(1, turnPageList[i-1]+1): # page = 1 ~ turnPage
        print("this is page: " + str(page) + "from total " + str(turnPageList[i-1]))
        offset = (page-1)*50  # offset: from index of hotel(in below url)
        url2 = "https://www.booking.com/searchresults.zh-tw.html?label=gen173nr-1DCAEoggJCAlhYSDNiBW5vcmVmaOcBiAEBmAEwuAEGyAEM2AED6AEBkgIBeagCAw&sid=4996667e149bfae490c17f2eb37eafa9&checkin_month="+str(checkinM[i-1])+"&checkin_monthday="+str(checkinD[i-1])+"&checkin_year="+str(checkinY[i-1])+"&checkout_month="+str(checkoutM[i-1])+"&checkout_monthday="+str(checkoutD[i-1])+"&checkout_year="+str(checkoutY[i-1])+"&class_interval=1&dest_id=-235402&dest_type=city&dtdisc=0&group_adults=2&group_children=0&inac=0&index_postcard=0&label_click=undef&no_rooms=1&postcard=0&raw_dest_type=city&room1=A%2CA&sb_price_type=total&src=index&src_elem=sb&ss="+str(location)+"&ss_all=0&ssb=empty&sshis=0&ssne="+str(location)+"&ssne_untouched="+str(location)+"&rows=50&offset="+ str(offset)### + "&sr_ajax=1&b_gtt=dLYAeZFVJfNTBdAHUdHPGZDfIfVNASUcbTYWPJLO&_=1501678454525"        
        res2 = requests.get(url2, headers = headers, cookies = cookie)
        soup2 = BeautifulSoup(res2.text, 'lxml')
        items = soup2.select(".sr_item")
        with open('BookingCrawl_now-{}-{}-{}-from-{}-to-{}-Page{}.txt'.format(time.localtime().tm_mon, time.localtime().tm_mday, time.localtime().tm_hour, timeList[i-1][0], timeList[i-1][1], page), 'w', encoding = 'UTF-8') as f:
            f.write(str(items))
            print("save done.")
            time.sleep(5)
            print("Cost time: " + str(time.time()-start_time))
            

period: 1
this is page: 1from total 3
save done.
Cost time: 7.476999759674072
this is page: 2from total 3
save done.
Cost time: 16.08399987220764
this is page: 3from total 3
save done.
Cost time: 24.372999906539917
period: 2
this is page: 1from total 3
save done.
Cost time: 31.931999921798706
this is page: 2from total 3
save done.
Cost time: 40.9229998588562
this is page: 3from total 3
save done.
Cost time: 49.359999895095825
period: 3
this is page: 1from total 3
save done.
Cost time: 57.14199995994568
this is page: 2from total 3
save done.
Cost time: 65.76699995994568
this is page: 3from total 3
save done.
Cost time: 74.9909999370575
period: 4
this is page: 1from total 3
save done.
Cost time: 82.3069999217987
this is page: 2from total 3
save done.
Cost time: 90.50300002098083
this is page: 3from total 3
save done.
Cost time: 98.5479998588562
period: 5
this is page: 1from total 4
save done.
Cost time: 105.36699986457825
this is page: 2from total 4
save done.
Cost time: 113.538000106811

save done.
Cost time: 731.7780001163483
this is page: 2from total 15
save done.
Cost time: 740.0049998760223
this is page: 3from total 15
save done.
Cost time: 748.8669998645782
this is page: 4from total 15
save done.
Cost time: 757.3659999370575
this is page: 5from total 15
save done.
Cost time: 766.492999792099
this is page: 6from total 15
save done.
Cost time: 774.7089998722076
this is page: 7from total 15
save done.
Cost time: 782.9409997463226
this is page: 8from total 15
save done.
Cost time: 791.2469999790192
this is page: 9from total 15
save done.
Cost time: 799.8519999980927
this is page: 10from total 15
save done.
Cost time: 808.5249998569489
this is page: 11from total 15
save done.
Cost time: 816.595999956131
this is page: 12from total 15
save done.
Cost time: 825.5060000419617
this is page: 13from total 15
save done.
Cost time: 833.4719998836517
this is page: 14from total 15
save done.
Cost time: 841.6569998264313
this is page: 15from total 15
save done.
Cost time: 849.8740

save done.
Cost time: 1602.805999994278
period: 21
this is page: 1from total 18
save done.
Cost time: 1610.8329997062683
this is page: 2from total 18
save done.
Cost time: 1619.9009997844696
this is page: 3from total 18
save done.
Cost time: 1628.0659999847412
this is page: 4from total 18
save done.
Cost time: 1636.5380001068115
this is page: 5from total 18
save done.
Cost time: 1645.1150000095367
this is page: 6from total 18
save done.
Cost time: 1653.2539999485016
this is page: 7from total 18
save done.
Cost time: 1661.923999786377
this is page: 8from total 18
save done.
Cost time: 1669.992999792099
this is page: 9from total 18
save done.
Cost time: 1678.095999956131
this is page: 10from total 18
save done.
Cost time: 1686.5129997730255
this is page: 11from total 18
save done.
Cost time: 1694.5359997749329
this is page: 12from total 18
save done.
Cost time: 1702.5409998893738
this is page: 13from total 18
save done.
Cost time: 1710.8069999217987
this is page: 14from total 18
save don

Cost time: 2448.1259999275208
this is page: 5from total 17
save done.
Cost time: 2456.8109998703003
this is page: 6from total 17
save done.
Cost time: 2464.9889998435974
this is page: 7from total 17
save done.
Cost time: 2473.2149999141693
this is page: 8from total 17
save done.
Cost time: 2481.34099984169
this is page: 9from total 17
save done.
Cost time: 2489.569999933243
this is page: 10from total 17
save done.
Cost time: 2498.0969998836517
this is page: 11from total 17
save done.
Cost time: 2506.177999973297
this is page: 12from total 17
save done.
Cost time: 2514.5460000038147
this is page: 13from total 17
save done.
Cost time: 2522.9189999103546
this is page: 14from total 17
save done.
Cost time: 2531.564999818802
this is page: 15from total 17
save done.
Cost time: 2539.572999715805
this is page: 16from total 17
save done.
Cost time: 2547.6639997959137
this is page: 17from total 17
save done.
Cost time: 2557.383999824524
period: 28
this is page: 1from total 18
save done.
Cost tim

save done.
Cost time: 3331.007999897003
this is page: 9from total 17
save done.
Cost time: 3339.742000102997
this is page: 10from total 17
save done.
Cost time: 3348.25
this is page: 11from total 17
save done.
Cost time: 3356.707999944687
this is page: 12from total 17
save done.
Cost time: 3364.9589998722076
this is page: 13from total 17
save done.
Cost time: 3373.470999956131
this is page: 14from total 17
save done.
Cost time: 3381.843999862671
this is page: 15from total 17
save done.
Cost time: 3390.1719999313354
this is page: 16from total 17
save done.
Cost time: 3398.5380001068115
this is page: 17from total 17
save done.
Cost time: 3407.5849997997284
period: 34
this is page: 1from total 18
save done.
Cost time: 3415.7129998207092
this is page: 2from total 18
save done.
Cost time: 3424.5
this is page: 3from total 18
save done.
Cost time: 3433.234999895096
this is page: 4from total 18
save done.
Cost time: 3441.7809998989105
this is page: 5from total 18
save done.
Cost time: 3450.453

save done.
Cost time: 4209.870000123978
this is page: 7from total 11
save done.
Cost time: 4218.27999997139
this is page: 8from total 11
save done.
Cost time: 4227.107999801636
this is page: 9from total 11
save done.
Cost time: 4236.270999908447
this is page: 10from total 11
save done.
Cost time: 4245.020999908447
this is page: 11from total 11
save done.
Cost time: 4253.658999919891
period: 41
this is page: 1from total 13
save done.
Cost time: 4261.411000013351
this is page: 2from total 13
save done.
Cost time: 4269.963000059128
this is page: 3from total 13
save done.
Cost time: 4278.774999856949
this is page: 4from total 13
save done.
Cost time: 4288.018000125885
this is page: 5from total 13
save done.
Cost time: 4296.71799993515
this is page: 6from total 13
save done.
Cost time: 4305.386999845505
this is page: 7from total 13
save done.
Cost time: 4313.867999792099
this is page: 8from total 13
save done.
Cost time: 4322.411999940872
this is page: 9from total 13
save done.
Cost time: 4

save done.
Cost time: 5106.3259999752045
this is page: 3from total 16
save done.
Cost time: 5115.701999902725
this is page: 4from total 16
save done.
Cost time: 5124.15499997139
this is page: 5from total 16
save done.
Cost time: 5133.446000099182
this is page: 6from total 16
save done.
Cost time: 5142.805999994278
this is page: 7from total 16
save done.
Cost time: 5151.359999895096
this is page: 8from total 16
save done.
Cost time: 5160.367999792099
this is page: 9from total 16
save done.
Cost time: 5170.046000003815
this is page: 10from total 16
save done.
Cost time: 5178.8140001297
this is page: 11from total 16
save done.
Cost time: 5187.462000131607
this is page: 12from total 16
save done.
Cost time: 5196.388999938965
this is page: 13from total 16
save done.
Cost time: 5204.732999801636
this is page: 14from total 16
save done.
Cost time: 5213.973999977112
this is page: 15from total 16
save done.
Cost time: 5222.426000118256
this is page: 16from total 16
save done.
Cost time: 5231.23

Cost time: 6015.946000099182
period: 54
this is page: 1from total 12
save done.
Cost time: 6024.508999824524
this is page: 2from total 12
save done.
Cost time: 6033.582999944687
this is page: 3from total 12
save done.
Cost time: 6042.834000110626
this is page: 4from total 12
save done.
Cost time: 6051.3099999427795
this is page: 5from total 12
save done.
Cost time: 6059.744999885559
this is page: 6from total 12
save done.
Cost time: 6068.468999862671
this is page: 7from total 12
save done.
Cost time: 6077.342000007629
this is page: 8from total 12
save done.
Cost time: 6086.1089997291565
this is page: 9from total 12
save done.
Cost time: 6094.506000041962
this is page: 10from total 12
save done.
Cost time: 6102.992999792099
this is page: 11from total 12
save done.
Cost time: 6112.046000003815
this is page: 12from total 12
save done.
Cost time: 6120.492999792099
period: 55
this is page: 1from total 12
save done.
Cost time: 6128.503000020981
this is page: 2from total 12
save done.
Cost ti

In [11]:
time.sleep(5)
print(3)

3
